#### Functions (IGNORE)

##### Packages

In [2]:
import pandas as pd
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt

##### Stochastic Modeling

In [3]:
from datetime import timedelta
def calculate_stock_roi(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment,future_investments, percent_to_buy, percent_to_sell):
    # Initialize a nested dictionary to store ROI percentages for each period and ticker
    roi_results = {period: {ticker: [] for ticker in tickers} for period in periods_list}

    # Loop through each economic period
    for period in periods_list:
        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=90))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=True)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            bb_signals_period = bb_signals_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]

            # Initialize variables
            account_balance = {ticker: future_investments for ticker in tickers}  # Separate account balance for each stock
            shares_number = {ticker: initial_investment/adj_close_period[ticker].iloc[0] for ticker in tickers}  # Initialize share count for each ticker
            shares_value = {ticker: initial_investment for ticker in tickers}   # Initialize share value for each ticker



            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]  # Correctly get ticker for each column
                    adj_close_price = adj_close_period.loc[row_idx, ticker]  # Get corresponding adjusted close price

                    # Handle Buy action
                    if signal == 'Buy':
                        amount_to_buy = percent_to_buy * account_balance[ticker]
                        if account_balance[ticker] >= amount_to_buy:
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy
                            account_balance[ticker] -= amount_to_buy

                    # Handle Sell action
                    elif signal == 'Sell':
                        if shares_number[ticker] > 0:
                            shares_value[ticker] = shares_number[ticker] * adj_close_price
                            amount_to_sell = percent_to_sell * shares_value[ticker]
                            if shares_value[ticker] >= amount_to_sell:
                                shares_to_sell = amount_to_sell / adj_close_price
                                shares_number[ticker] -= shares_to_sell
                                account_balance[ticker] += amount_to_sell

            # Calculate total portfolio value for each stock at the end of the period
            for ticker in tickers:
                if shares_number[ticker] > 0:  # Only calculate value if shares are owned
                    portfolio_value = shares_number[ticker] * adj_close_period.iloc[-1][ticker]
                    total_value = account_balance[ticker] + portfolio_value
                    
                    # Calculate profit for this stock
                    profit = total_value - (initial_investment+future_investments)
                    
                    # Calculate ROI based on stock's individual account
                    roi_dollar_value = profit / (initial_investment+future_investments) * 100
                else:
                    roi_dollar_value = 0

                # Store ROI in the results dictionary
                roi_results[period][ticker].append(roi_dollar_value)

    return roi_results

In [4]:
from datetime import timedelta

# create a list of signal pages
signals_list = ['bollinger_bands','rsi_signals','ichimoku_signals','macd_signals']

# create an all_data functions
def combined_data(adj_close_data,signals_list,periods,tickers):
    # create an all data dictionary where each period is a list
    all_data = {period: {} for period in periods}
    for period in periods:
        # add to all data with different periods in different signals with each ticker as the columns
        all_data[period] = {signal: pd.DataFrame(columns=tickers) for signal in signals_list}

        # add the adjusted close sector etf data
        all_data[period]['adj_close'] = adj_close_data[period]

            # add the different signals data
        for signal in signals_list:
            all_data[period][signal] = signals_dict[period][signal]
    
    return all_data


In [5]:
def create_features(data, periods, tickers):
    """
    Adds Bollinger Bands, MACD, and RSI signals to the data for each period and ticker.
    The signals are:
    - 1 for Buy
    - -1 for Sell
    - 0 for Hold
    """
    for period in periods:
        for ticker in tickers:
            # Ensure the features are calculated for each period and ticker
            df = data[period][ticker]
            
            # Bollinger Bands Signal
            df['Bollinger_Signal'] = np.where(df['Adj Close'] < df['lower_band'], 1, 
                                              np.where(df['Adj Close'] > df['upper_band'], -1, 0))

            # MACD Signal
            df['MACD_Signal'] = np.where(df['macd_line'] > df['signal_line'], 1, 
                                         np.where(df['macd_line'] < df['signal_line'], -1, 0))

            # RSI Signal
            df['RSI_Signal'] = np.where(df['RSI'] < 30, 1, 
                                        np.where(df['RSI'] > 70, -1, 0))

            # Store the updated DataFrame back into the dictionary
            data[period][ticker] = df

    return data

In [6]:
import pandas as pd

def collect_signals(data, list_techniques, tickers, periods):
    # Initialize a nested dictionary for signals
    signals_nd = {period: {technique: pd.DataFrame(columns=tickers) for technique in list_techniques} for period in periods}
    
    for period in periods:
        for technique in list_techniques:
            for ticker in tickers:
                # Check if the period and ticker exist in the data
                if period in data and ticker in data[period]:
                    # Check if the technique's signal exists
                    signal_column = f'{technique}_Signal'
                    if signal_column in data[period][ticker].columns:
                        # Collect the signals for the specified period, technique, and ticker
                        signals_nd[period][technique][ticker] = data[period][ticker][signal_column]
                    else:
                        # Handle the case where the signal column doesn't exist
                        signals_nd[period][technique][ticker] = pd.Series([None] * len(data[period][ticker]), index=data[period][ticker].index)
                else:
                    # Handle missing period or ticker data
                    signals_nd[period][technique][ticker] = pd.Series([None] * len(data[period][ticker]), index=data[period][ticker].index)

    return signals_nd


##### Collect Data

In [7]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [8]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [9]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

##### Folder Load in

In [10]:
import os
import pandas as pd

# Define the root directory
root_dir = "/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Financial Capstone Project/data/signal_data"

# Initialize an empty dictionary
signals_dict = {}

# Loop through the directories and files
for foldername, subfolders, filenames in os.walk(root_dir):
    # Get the signal type from the folder name (last part of the path)
    signal_type = os.path.basename(foldername)
    
    # Loop through the files
    for filename in filenames:
        # Check if the file is a CSV
        if filename.endswith('.csv'):
            # Extract the period from the file name (e.g., "expansion", "contraction", etc.)
            period = filename.split('_')[0]  # Assuming period is the first part of the file name

            # Initialize the dictionary for the period if not already present
            if period not in signals_dict:
                signals_dict[period] = {}

            # Build the full file path
            file_path = os.path.join(foldername, filename)
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Add the DataFrame to the nested dictionary under the corresponding period and signal type
            signals_dict[period][signal_type] = df.set_index('Date')

##### Bollinger Bands

In [11]:
# create bollinger bands
import scipy.stats as stats
def add_bollinger_data(data,window,conf_int):
        z_score = stats.norm.ppf(1 - (1 - conf_int) / 2) # create a zscore from the mean

        data['middle_band'] = data['Adj Close'].rolling(window).mean()
        data['upper_band'] = data['middle_band'] + z_score * data['Adj Close'].rolling(window).std()
        data['lower_band'] = data['middle_band'] - z_score * data['Adj Close'].rolling(window).std()

        data['Signal'] = 'Hold'

        data['Signal'] = np.where(data['Adj Close'] < data['lower_band'], 'Buy', 
                              np.where(data['Adj Close'] > data['upper_band'], 'Sell', 'Hold'))

        return data

In [12]:
# create bollinger data for multiple time period and multiple tickers
def bollinger_data_multiple_periods_tickers(periods,tickers,data,window,confidence_period):
    # for each ticker in economic time periods
    for period in periods:
            for ticker in tickers:
                    try:
                        add_bollinger_data(data[period][ticker],window,confidence_period)
                    except KeyError:
                        print(f'Data for {ticker} does not exist during {period}')
    return data

##### RSI

In [13]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a given stock data series.

    Parameters:
    data (pd.Series): A pandas series of adjusted close prices.
    window (int): The lookback period for RSI calculation, default is 14.

    Returns:
    pd.Series: RSI values.
    """
    delta = data.diff()  # Difference in price from previous price
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()  # Average gain
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()  # Average loss
    
    # Avoid division by zero, especially at the beginning of the dataset
    rs = gain / loss.replace(0, np.nan)  
    
    # RSI formula
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [14]:
# create rsi value in sector etf dataframe

def rsi_value(nested_dict,periods,tickers):
    for period in periods:
        for ticker in tickers:
            nested_dict[period][ticker]['RSI'] = calculate_rsi(nested_dict[period][ticker]['Adj Close'])

    return nested_dict

In [15]:
import numpy as np

def create_rsi_signal(nested_dict, periods, tickers):
    """
    Adds a 'Signal' column to the nested dictionary based on RSI values.

    Parameters:
    - nested_dict: A nested dictionary where each period contains dataframes for tickers.
                   Each dataframe should have an 'RSI' column.
    - periods: A list of periods to iterate over.
    - tickers: A list of tickers to process within each period.

    Returns:
    - The modified nested dictionary with new 'Signal' columns.
    """
    
    for period in periods:
        for ticker in tickers:
            # Create the 'Signal' column using np.where
            nested_dict[period][ticker]['Signal'] = np.where(
                nested_dict[period][ticker]['RSI'] < 30, 'Buy',
                np.where(nested_dict[period][ticker]['RSI'] > 70, 'Sell', 'Hold')
            )

    return nested_dict

##### MACD 

In [16]:
def macd_components(nested_dict,periods,tickers):
    for period in periods:
        for ticker in tickers:
            # get the short and long ema
            nested_dict[period][ticker]['short_ema'] = nested_dict[period][ticker]['Close'].ewm(span=12, adjust=False).mean()
            nested_dict[period][ticker]['long_ema'] = nested_dict[period][ticker]['Close'].ewm(span=26, adjust=False).mean()

            # create the MACD line
            nested_dict[period][ticker]['macd_line'] = nested_dict[period][ticker]['short_ema'] - nested_dict[period][ticker]['long_ema']
            nested_dict[period][ticker]['signal_line'] = nested_dict[period][ticker]['macd_line'].ewm(span=9, adjust=False).mean()

    return nested_dict

In [17]:
def add_macd_signals(nested_dict,periods, tickers, short_window=12, long_window=26, signal_window=9):
    """
    Function to calculate MACD, Signal Line, and generate Buy/Sell/Hold signals based on crossovers.
    """
    for period in periods:
        for ticker in tickers:

            data = nested_dict[period][ticker]

            # Generate 'Buy', 'Sell', 'Hold' signals using np.where() based on crossovers
            data['Signal'] = np.where(data['macd_line'] > data['signal_line'], 'Buy', 
                                    np.where(data['macd_line'] < data['signal_line'], 'Sell', 'Hold'))

In [18]:
def generate_signals(nested_dict, periods, tickers):
    for period in periods:
        for ticker in tickers:
            # Create lists to store buy and sell signals
            buy_signals = []
            sell_signals = []

            # Initialize variables to track the last state (to detect crossovers)
            last_macd_above_signal = None

            # Loop through each row in the MACD dataframe
            for i in range(len(nested_dict[period][ticker]['macd_line'])):
                macd_line = nested_dict[period][ticker]['macd_line'].iloc[i]
                signal_line = nested_dict[period][ticker]['signal_line'].iloc[i]

                # Determine buy/sell signal based on MACD crossover
                if macd_line > signal_line and (last_macd_above_signal is False or last_macd_above_signal is None):
                    buy_signals.append(nested_dict[period][ticker]['Close'].iloc[i])
                    sell_signals.append(None)
                    last_macd_above_signal = True
                elif macd_line < signal_line and (last_macd_above_signal is True or last_macd_above_signal is None):
                    sell_signals.append(nested_dict[period][ticker]['Close'].iloc[i])
                    buy_signals.append(None)
                    last_macd_above_signal = False
                else:
                    # No signal
                    buy_signals.append(None)
                    sell_signals.append(None)

            # Store buy and sell signals in the nested dictionary
            nested_dict[period][ticker]['buy_signals'] = buy_signals
            nested_dict[period][ticker]['sell_signals'] = sell_signals



### Load in Data

In [19]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
    'all_data': ('2005-01-01','2024-06-01')
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction','all_data']

In [20]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

In [21]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [22]:
adj_close_data = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

## Technical Analysis Strategies Signal Creations

### Create Bollinger Bands

In [23]:
# create the data for bollinger bands
sector_etf_data = bollinger_data_multiple_periods_tickers(economic_cycle_periods_list,sector_etf_tickers,sector_etf_data,30,0.90)

### Create MACD Signal

In [24]:
# create the data for macd
sector_etf_data = macd_components(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

### Create RSI Signal

In [25]:
# create the data for rsi
sector_etf_data = rsi_value(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [26]:
sector_etf_data = create_features(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [27]:
sector_etf_data['trough']['XLB']

,Open,High,Low,Close,Adj Close,Volume,middle_band,upper_band,lower_band,Signal,short_ema,long_ema,macd_line,signal_line,RSI,Bollinger_Signal,MACD_Signal,RSI_Signal
Date,,,,,,,,,,,,,,,,,,
2008-10-01,32.759998,33.189999,32.130001,32.849998,23.119259,14639500,NaN,NaN,NaN,Hold,32.849998,32.849998,0.000000,0.000000,NaN,0,0,0
2008-10-02,31.540001,31.860001,29.930000,30.490000,21.458338,12581300,NaN,NaN,NaN,Hold,32.486922,32.675184,-0.188262,-0.037652,NaN,0,-1,0
2008-10-03,30.190001,31.690001,29.780001,30.190001,21.247198,16770600,NaN,NaN,NaN,Hold,32.133549,32.491096,-0.357547,-0.101631,NaN,0,-1,0
2008-10-06,29.510000,29.510000,26.889999,28.700001,20.198557,22512700,NaN,NaN,NaN,Hold,31.605311,32.210274,-0.604963,-0.202298,NaN,0,-1,0
2008-10-07,29.160000,29.530001,27.049999,27.219999,19.156969,16004900,NaN,NaN,NaN,Hold,30.930648,31.840624,-0.909977,-0.343833,NaN,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-05-22,26.530001,26.660000,26.110001,26.299999,18.848713,8421500,18.249808,19.693871,16.805745,Hold,26.303890,25.628573,0.675318,0.792434,42.280671,0,-1,0
2009-05-26,26.170000,26.969999,25.830000,26.930000,19.300224,7886900,18.316220,19.762987,16.869454,Hold,26.400215,25.724975,0.675240,0.768995,49.270695,0,-1,0
2009-05-27,26.790001,26.850000,25.860001,25.920000,18.576382,7216600,18.369255,19.757178,16.981332,Hold,26.326336,25.739421,0.586915,0.732579,41.442779,0,-1,0


In [28]:
techniques_list = ['Bollinger','MACD','RSI']
buy_sell_signals = collect_signals(sector_etf_data,techniques_list,sector_etf_tickers,economic_cycle_periods_list)

In [30]:
def add_signals(periods, technique_weights, tickers, technique_signals):
    # Initialize signals dictionary with DataFrames of zeros for each period
    signals = {period: pd.DataFrame(0, columns=tickers, index=technique_signals[period]['Bollinger'].index) 
               for period in periods}
    
    # Loop through each period
    for period in periods:
        for technique, weight in technique_weights.items():
            # Access the technique signals for the current period
            df = technique_signals[period][technique]

            # Ensure that the signal value is either 0 or the weighted value
            weighted_df = df * weight

            # Add the weighted signal for each ticker, ensuring alignment and presence of data
            signals[period] += weighted_df
    
    return signals

# Call the function
signals_result = add_signals(economic_cycle_periods_list, weights, sector_etf_tickers, buy_sell_signals)
signals_result

{'trough':             XLB  XLI  XLF  XLK  XLY  XLP  XLE  XLV  VOX  XLU  IYR
 Date                                                             
 2008-10-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2008-10-02 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4  0.4 -0.4 -0.4 -0.4
 2008-10-03 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 2008-10-06 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 2008-10-07 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 ...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
 2009-05-22 -0.4 -0.4 -0.4 -0.4 -0.4  0.4 -0.4  0.4 -0.4 -0.4 -0.4
 2009-05-26 -0.4 -0.4 -0.4 -0.4 -0.4  0.0 -0.4 -0.4 -0.4 -0.4 -0.4
 2009-05-27 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 2009-05-28 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 2009-05-29 -0.4 -0.4 -0.4  0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4 -0.4
 
 [166 rows x 11 columns],
 'expansion':             XLB  XLI  XLF  XLK  XLY  XLP  XLE  XLV  VOX  XLU  IYR
 Date      

In [31]:
def weighted_technique_roi(technique_weights, signals_nd,adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment,future_investments, percent_to_buy, percent_to_sell):
    # Initialize a nested dictionary to store ROI percentages for each period and ticker
    roi_results = {period: {ticker: [] for ticker in tickers} for period in periods_list}
    signals = {period: pd.DataFrame(0, columns=sector_etf_tickers, index=pd.to_datetime(buy_sell_signals[period]['Bollinger'].index)) 
        for period in economic_cycle_periods_list}
    

    # Loop through each economic period
    for period in periods_list:
        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=90))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=True)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            for technique, weight in technique_weights.items():
                # Access the technique signals for the current period
                df = signals_nd[period][technique]

                # Ensure that the signal value is either 0 or the weighted value
                weighted_df = df * weight

                # Add the weighted signal for each ticker, ensuring alignment and presence of data
                signals[period] += weighted_df

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            signals_period = signals[period].loc[time_stamp:time_stamp + timedelta(days=90)]

            # Initialize variables
            account_balance = {ticker: future_investments for ticker in tickers}  # Separate account balance for each stock
            shares_number = {ticker: initial_investment/adj_close_period[ticker].iloc[0] for ticker in tickers}  # Initialize share count for each ticker
            shares_value = {ticker: initial_investment for ticker in tickers}   # Initialize share value for each ticker

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in signals_period.iterrows():
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]  # Correctly get ticker for each column
            
                    adj_close_price = adj_close_period.loc[row_idx, ticker]  # Get corresponding adjusted close price

                    # Handle Buy action
                    if signal > 0:
                        amount_to_buy = percent_to_buy * account_balance[ticker]
                        if account_balance[ticker] >= amount_to_buy:
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy
                            account_balance[ticker] -= amount_to_buy

                    # Handle Sell action
                    elif signal < 0:
                        if shares_number[ticker] > 0:
                            shares_value[ticker] = shares_number[ticker] * adj_close_price
                            amount_to_sell = percent_to_sell * shares_value[ticker]
                            if shares_value[ticker] >= amount_to_sell:
                                shares_to_sell = amount_to_sell / adj_close_price
                                shares_number[ticker] -= shares_to_sell
                                account_balance[ticker] += amount_to_sell

            # Calculate total portfolio value for each stock at the end of the period
            for ticker in tickers:
                if shares_number[ticker] > 0:  # Only calculate value if shares are owned
                    portfolio_value = shares_number[ticker] * adj_close_period.iloc[-1][ticker]
                    total_value = account_balance[ticker] + portfolio_value
                    
                    # Calculate profit for this stock
                    profit = total_value - (initial_investment+future_investments)
                    
                    # Calculate ROI based on stock's individual account
                    roi_dollar_value = profit / (initial_investment+future_investments) * 100
                else:
                    roi_dollar_value = 0

                # Store ROI in the results dictionary
                roi_results[period][ticker].append(roi_dollar_value)

    return roi_results

In [32]:
def stochastic_roi(tickers,periods,return_rates_list,analysis_type):
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = pd.Series(return_rates_list[period][ticker])
            if analysis_type=='Mean':
                df.at[ticker,period] = data.mean()
            elif analysis_type=='Median':
                df.at[ticker,period] = data.median()
            elif analysis_type=='Std':
                df.at[ticker,period] = data.std()
            elif analysis_type=='Variance':
                df.at[ticker,period] = data.var()

    return df

In [33]:
weights  = {
    'Bollinger': 0.4,
    'RSI': 0.2,
    'MACD': 0.4
}
a= weighted_technique_roi(weights,buy_sell_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,100,0,100,0.2,0.2)

In [46]:
weighted_technique = stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,a,'Mean')

In [ ]:
weighted_technique.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Financial Capstone Project/data/performance_data/multiple_signals/weighted.csv')

In [35]:
from itertools import product
def grid_search_weighted_technique_roi(technique_weights_grid, signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment, future_investments, percent_to_buy, percent_to_sell):
    # Store results for each weight combination
    results = []
    
    # Iterate over combinations of weights
    for weights in product(*technique_weights_grid.values()):
        weight_dict = dict(zip(technique_weights_grid.keys(), weights))
        
        # Calculate ROI for the current combination of weights
        roi_results = weighted_technique_roi(weight_dict, signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment, future_investments, percent_to_buy, percent_to_sell)
        
        # Average the ROI across all periods and tickers
        average_roi = np.mean([np.mean(roi_results[period][ticker]) for period in periods_list for ticker in tickers])
        
        # Append the result (weights and average ROI) to the results list
        results.append({
            'weights': weight_dict,
            'average_roi': average_roi
        })
    
    # Convert the results list to a DataFrame
    roi_df = pd.DataFrame(results)
    
    return roi_df

In [36]:
grid_weights  = {
    'Bollinger': [0.2,0.4,0.6,0.8],
    'RSI': [0.2,0.4,0.6,0.8],
    'MACD':  [0.2,0.4,0.6,0.8]
}
roi = grid_search_weighted_technique_roi(grid_weights,buy_sell_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,100,0,100,0.20,0.2)

In [37]:
roi.sort_values(by='average_roi',ascending=False)

,weights,average_roi
62,"{'Bollinger': 0.8, 'RSI': 0.8, 'MACD': 0.6}",1.962093
44,"{'Bollinger': 0.6, 'RSI': 0.8, 'MACD': 0.2}",1.932438
40,"{'Bollinger': 0.6, 'RSI': 0.6, 'MACD': 0.2}",1.862947
47,"{'Bollinger': 0.6, 'RSI': 0.8, 'MACD': 0.8}",1.802168
53,"{'Bollinger': 0.8, 'RSI': 0.4, 'MACD': 0.4}",1.795015
...,...,...
2,"{'Bollinger': 0.2, 'RSI': 0.2, 'MACD': 0.6}",0.458577
19,"{'Bollinger': 0.4, 'RSI': 0.2, 'MACD': 0.8}",0.391919
11,"{'Bollinger': 0.2, 'RSI': 0.6, 'MACD': 0.8}",0.391556
3,"{'Bollinger': 0.2, 'RSI': 0.2, 'MACD': 0.8}",0.311903


In [40]:
weights  = {
    'Bollinger': 0.8,
    'RSI': 0.8,
    'MACD': 0.6
}
optimal = weighted_technique_roi(weights,buy_sell_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,1000,0,100,0.2,0.2)

In [44]:
optimal_weight = stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,optimal,'Mean')

In [45]:
optimal_weight.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Financial Capstone Project/data/performance_data/multiple_signals/optimal_weights.csv')

In [43]:
stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,optimal,'Mean').mean()

trough         1.059383
expansion      1.912081
peak           3.134041
contraction   -0.427363
all_data       1.565281
dtype: float64